# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
df = pd.read_csv('Cities_Weather_Data.csv')

df


,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,chuy,-64.557276,-25.014928,67.26,54,3,9.66
1,khandyga,62.988479,135.507358,45.09,66,96,1.28
2,necochea,-45.158673,-57.257843,71.04,49,88,19.53
3,maniitsoq,61.574943,-56.845925,35.96,87,61,2.51
4,qaanaaq,74.862936,-85.096441,36.88,82,49,4.21
...,...,...,...,...,...,...,...
542,broome,-18.012177,122.982997,62.64,62,75,3.00
543,lander,42.108690,-108.329389,60.13,48,1,0.00
544,las palmas,28.059591,-15.399694,70.59,56,40,10.67
545,ocosingo,16.898675,-91.523067,81.64,47,62,4.74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
df['Humidity (%)'].max()

100

In [48]:
locations = df[['Latitude','Longitude']]
humidity = df['Humidity (%)']
# configure gmaps
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
new_df = df[(df['Temperature (F)']>65) & (df['Temperature (F)']<=85) & (df['Cloudiness (%)']<10) & (df['Wind Speed (mph)']<10)]


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
hotel_df = new_df.copy()
hotel_df

,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,chuy,-64.557276,-25.014928,67.26,54,3,9.66
6,albany,-70.525071,100.987161,68.54,53,4,5.99
34,bucerias,20.330817,-106.624814,84.18,58,1,8.19
66,tautira,-15.675652,-144.641337,77.02,73,4,6.80
70,fare,-10.606512,-147.993263,80.29,72,4,8.14
88,banjar,-12.077211,105.966397,80.46,90,8,2.30
90,cidreira,-32.702088,-46.767089,69.75,49,1,8.41
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19
130,makakilo city,13.734640,-161.087986,73.92,75,1,3.44
135,montepuez,-12.923175,38.635623,65.32,54,8,3.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
hotel_df['Hotel Name'] = ""
hotel_df

,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,chuy,-64.557276,-25.014928,67.26,54,3,9.66,
6,albany,-70.525071,100.987161,68.54,53,4,5.99,
34,bucerias,20.330817,-106.624814,84.18,58,1,8.19,
66,tautira,-15.675652,-144.641337,77.02,73,4,6.80,
70,fare,-10.606512,-147.993263,80.29,72,4,8.14,
88,banjar,-12.077211,105.966397,80.46,90,8,2.30,
90,cidreira,-32.702088,-46.767089,69.75,49,1,8.41,
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19,
130,makakilo city,13.734640,-161.087986,73.92,75,1,3.44,
135,montepuez,-12.923175,38.635623,65.32,54,8,3.80,


In [52]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "type": "lodging",
    "radius": 5000, 
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    city = row['Nearest City']
    lat = row['Latitude']
    lng = row['Longitude']
    params['location']= f"{lat},{lng}"
    print(f"retrieving results for {index}:{lat} & {lng}")
    response = requests.get(url, params=params).json()
    
    
    try: 
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
    except (KeyError, IndexError):
        print(f"Missing field for {index}: {city}...skipping")


retrieving results for 0:-64.55727558266453 & -25.014927530704995
Missing field for 0: chuy...skipping
retrieving results for 6:-70.5250711349194 & 100.98716143304632
Missing field for 6: albany...skipping
retrieving results for 34:20.3308167597287 & -106.62481389964456
Missing field for 34: bucerias...skipping
retrieving results for 66:-15.675652157831266 & -144.64133713863595
Missing field for 66: tautira...skipping
retrieving results for 70:-10.606511780081973 & -147.9932634488599
Missing field for 70: fare...skipping
retrieving results for 88:-12.07721071026114 & 105.966396999883
Missing field for 88: banjar...skipping
retrieving results for 90:-32.702087643203626 & -46.76708912384215
Missing field for 90: cidreira...skipping
retrieving results for 104:-33.210172092454144 & -60.31677253940694
retrieving results for 130:13.73464024706017 & -161.08798557923618
Missing field for 130: makakilo city...skipping
retrieving results for 135:-12.92317451395364 & 38.63562312182577
Missing fie

In [53]:
hotel_df

,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,chuy,-64.557276,-25.014928,67.26,54,3,9.66,
6,albany,-70.525071,100.987161,68.54,53,4,5.99,
34,bucerias,20.330817,-106.624814,84.18,58,1,8.19,
66,tautira,-15.675652,-144.641337,77.02,73,4,6.80,
70,fare,-10.606512,-147.993263,80.29,72,4,8.14,
88,banjar,-12.077211,105.966397,80.46,90,8,2.30,
90,cidreira,-32.702088,-46.767089,69.75,49,1,8.41,
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19,HOTEL EMBAJADOR
130,makakilo city,13.734640,-161.087986,73.92,75,1,3.44,
135,montepuez,-12.923175,38.635623,65.32,54,8,3.80,


In [54]:
hotel_df[(hotel_df['Hotel Name']!="") & (hotel_df['Temperature (F)']>60) & (hotel_df['Temperature (F)']<80) ]

,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19,HOTEL EMBAJADOR
325,sari,36.640329,53.055858,67.59,88,0,4.90,Narenj Hotel
515,kamyaran,34.847341,46.860804,71.55,32,0,1.66,باغ فرامرز کمانگر
517,tooele,39.992089,-112.421527,68.56,28,1,2.13,Vernon Reservoir Campground


In [55]:
perfect_list = hotel_df[(hotel_df['Hotel Name']!="") & (hotel_df['Temperature (F)']>60) & (hotel_df['Temperature (F)']<80) ]

In [56]:

perfect_list

,Nearest City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19,HOTEL EMBAJADOR
325,sari,36.640329,53.055858,67.59,88,0,4.90,Narenj Hotel
515,kamyaran,34.847341,46.860804,71.55,32,0,1.66,باغ فرامرز کمانگر
517,tooele,39.992089,-112.421527,68.56,28,1,2.13,Vernon Reservoir Campground


In [57]:
perfect_list.rename(columns={'Nearest City': 'City'}, inplace=True)
perfect_list

C:\Users\sreek\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
104,villa constitucion,-33.210172,-60.316773,73.38,49,0,8.19,HOTEL EMBAJADOR
325,sari,36.640329,53.055858,67.59,88,0,4.90,Narenj Hotel
515,kamyaran,34.847341,46.860804,71.55,32,0,1.66,باغ فرامرز کمانگر
517,tooele,39.992089,-112.421527,68.56,28,1,2.13,Vernon Reservoir Campground


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_list.iterrows()]
locations = perfect_list[["Latitude", "Longitude"]]

In [59]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))